- 유튜브랭킹

In [116]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
from urllib.parse import quote
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta

In [117]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')


In [118]:
result = requests.get(url+str(1))
soup = BeautifulSoup(result.text, 'html.parser')
lis = soup.select('form > table > tbody> tr')

In [119]:
len(lis)
rank = lis[0].select_one('.rank')
rank

<td class="rank">
                        1                    </td>

In [120]:
# 자동으로 해줌
driver.get(url+str(2))
body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [121]:
soup = BeautifulSoup(driver.page_source)

In [122]:
driver.close()

In [123]:
lis = soup.select('.aos-init')
len(lis)

100

In [124]:
li = lis[0]

In [125]:
# rank
rank = li.select_one('.rank').get_text().strip()
rank

'102'

In [126]:
# category
category = li.select_one('.category').get_text().strip()
category

'[TV/방송]'

In [127]:
# channel 명
channel = li.select_one('h1 > a').get_text().strip()
channel

'JTBC Drama'

In [128]:
# 구독자
subscriber = li.select_one('.subscriber_cnt').get_text().strip()
subscriber = subscriber.replace('만', '0000')
subscriber

'4730000'

In [129]:
# view 순
view = li.select_one('.view_cnt').get_text().strip()
view = view.replace('억', '').replace('만','0000')
view

'5238230000'

In [130]:
# video 순
video = li.select_one('.video_cnt').get_text().strip()
video = video.replace('개', '')
video


'27,755'

In [131]:
# 숫자로 바꿔주는 함수
import re

def convert_num(s):
    s = re.sub('[,억개 ]', '', s).replace('만', '0000')
    return int(s)

In [132]:
# import re
# re.sub('[개,]', '', )
lines = []
for li in lis:
    rank = int(li.select_one('.rank').get_text().strip())
    category = li.select_one('.category').get_text().strip().strip('[]')
    channel = li.select_one('h1 > a').get_text().strip()
    
    subscriber = convert_num(li.select_one('.subscriber_cnt').get_text())
    view = convert_num(li.select_one('.view_cnt').get_text())
    video = convert_num(li.select_one('.video_cnt').get_text())

    lines.append({'순위': rank, '카테고리': category, '채널': channel, '구독자수': subscriber, '조회수': view, '비디오': video})

df = pd.DataFrame(lines)  
df.head()   

,순위,카테고리,채널,구독자수,조회수,비디오
0,102,TV/방송,JTBC Drama,4730000,5238230000,27755
1,101,음식/요리/레시피,까니짱 [ G-NI : 蟹ちゃん],4730000,1395430000,763
2,103,게임,EA SPORTS FIFA,4570000,842240000,803
3,105,음악/댄스/가수,STUDIO CHOOM [스튜디오 춤],4470000,2433290000,1460
4,104,TV/방송,채널 NCT DAILY,4470000,849730000,467


In [133]:
df.tail()

,순위,카테고리,채널,구독자수,조회수,비디오
95,196,음식/요리/레시피,한세 HANSE,2700000,446510000,675
96,197,해외,The Wiggles,2670000,3615310000,798
97,198,교육/강의,슈카월드,2660000,682060000,1370
98,199,취미/라이프,ASMR PPOMO,2640000,792780000,590
99,200,BJ/인물/연예인,진용진,2630000,930860000,538


- selenuim 쓰면서 가져오기

In [137]:
curl = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')
lines = []

for i in range(1, 11):
    
    driver.get(curl + str(i))
    time.sleep(2)   # 가져오는데 시간이 필요하다

    # # 자동으로 해줌
    # body = driver.find_element(By.TAG_NAME, 'body')
    # for _ in range(15):
    #     body.send_keys(Keys.PAGE_DOWN)
    #     time.sleep(1)

    soup = BeautifulSoup(driver.page_source)
    lis = soup.select('.aos-init')

    for li in lis:
        rank = int(li.select_one('.rank').get_text().strip())
        category = li.select_one('.category').get_text().strip()[1:-1]
        channel = li.select_one('h1 > a').get_text().strip()

        subscriber = convert_num(li.select_one('.subscriber_cnt').get_text())
        view = convert_num(li.select_one('.view_cnt').get_text())
        video = convert_num(li.select_one('.video_cnt').get_text())

        lines.append({'랭크': rank, '카테고리': category, '채널': channel, '구독자수': subscriber, '조회수': view, '비디오': video})

driver.close()

In [138]:
df = pd.DataFrame(lines)  
df.head()  

,랭크,카테고리,채널,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,87300000,30533500000,507
1,2,음악/댄스/가수,BANGTANTV,74500000,20143000000,2188
2,3,음악/댄스/가수,HYBE LABELS,70300000,27323160000,1158
3,4,음악/댄스/가수,SMTOWN,31600000,26794810000,4110
4,5,음악/댄스/가수,JYP Entertainment,26800000,19275070000,1668


In [139]:
now = datetime.now()
now = now.strftime('%Y%m%d')
df.to_csv(f'data/{now}_유튜브랭킹.csv', index=False)